In [1]:
from pymongo import MongoClient
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import re
import datetime

%matplotlib inline

In [2]:
#ipclient = Client()
#print (ipclient.ids)
#ipview = ipclient[:]
#ipview = ipclient.load_balanced_view()

In [3]:
client = MongoClient()
db = client.VentDyssynchrony_db
log_data = db.LogData_collection
patient_data = db.PatientData_collection

## ID FileTypes for Each Log File

In [4]:
log = log_data.find()

for items in log:
    results = re.search('((?<=hm_)[a-zA-Z]+)|(edit[a-zA-Z]+)|(Breath)|(Waveform)', items['file_name'])
    
    if results:
       log_data.update({'_id':items['_id']},
                       {'$set': {'file_type':results.group(0) }})

    

## Number of Log Files

In [5]:
log_data.find().count()

515

In [6]:
files = log_data.aggregate([{'$group':{'_id':'$file_type', 'number_files':{'$sum':1}, 
                               'patients':{'$addToSet':'$patientID'}}}])
file_log = pd.DataFrame(list(files))

## Create List of Patients who Are Missing FileTypes

In [7]:
patient_list = log_data.aggregate([{'$match':{'file_type':'Waveform'}},
                    {'$group':{'_id':'$file_type', 'patients':{'$addToSet':'$patientID'}}}])
patient_list =set(list(patient_list)[0]['patients'])

print(sorted(patient_list))
print(len(patient_list))


['P100', 'P101', 'P102', 'P103', 'P104', 'P105', 'P106', 'P107', 'P108', 'P109', 'P110', 'P112', 'P113', 'P114', 'P115', 'P116', 'P117', 'P118', 'P119', 'P120', 'P121', 'P122']
22


In [8]:
file_log['patient_count'] = file_log['patients'].apply(lambda x: len(x))
file_log['missing_patients'] = file_log['patients'].apply(lambda x: sorted(list(patient_list - set(x))))
file_log

,_id,number_files,patients,patient_count,missing_patients
0,editedRN,21,"[P121, P120, P118, P117, P100, P116, P115, P11...",21,[P122]
1,editLab,21,"[P121, P120, P118, P117, P100, P116, P115, P11...",21,[P122]
2,Breath,227,"[P121, P120, P117, P100, P115, P116, P118, P11...",21,[P104]
3,editedRT,21,"[P121, P120, P118, P117, P110, P114, P105, P10...",21,[P122]
4,Waveform,225,"[P121, P120, P118, P117, P100, P116, P115, P11...",22,[]


## Create Table of Recording Time for Each Patient

In [9]:
elapse = log_data.aggregate([{'$match': {'file_type':'Waveform'}},
                    {'$project':{'patientID':1, 'end_time':1, 'start_time':1, 'elapse_time':{'$subtract':['$end_time', '$start_time']}}},                  
                    {'$group': {'_id': '$patientID', 'total_time':{'$sum':'$elapse_time'}, 
                                'number_files':{'$sum':1}}},
                    {'$project':{'_id':1, 'number_files':1, 'total_hours':{'$divide':['$total_time', 1000*60*60]}}},
                    {'$sort':{'total_hours':1}}])

In [10]:
elapsed_recording = pd.DataFrame(list(elapse))
elapsed_recording.sort(columns='_id')

,_id,number_files,total_hours
4,P100,3,5.355556
13,P101,8,37.100278
1,P102,3,1.029167
11,P103,5,33.395000
0,P104,1,0.000000
10,P105,6,30.086667
12,P106,10,36.621111
3,P107,2,4.404444
2,P108,1,2.431667
8,P109,11,26.845278


In [11]:
elapsed_recording.describe()

,number_files,total_hours
count,22.000000,22.000000
mean,10.227273,42.713763
std,8.613012,41.970874
min,1.000000,0.000000
25%,3.500000,12.054444
50%,8.500000,31.740833
75%,11.750000,51.299792
max,35.000000,157.390278


In [12]:
patient_data.find().count()

21

In [13]:
elapse = log_data.aggregate([{'$match': {'file_type':'Breath'}},
                    {'$project':{'patientID':1, 'end_time':1, 'start_time':1, 'elapse_time':{'$subtract':['$end_time', '$start_time']}}},                  
                    {'$group': {'_id': '$patientID', 'total_time':{'$sum':'$elapse_time'}, 
                                'number_files':{'$sum':1}}},
                    {'$project':{'_id':1, 'number_files':1, 'total_hours':{'$divide':['$total_time', 1000*60*60]}}},
                    {'$sort':{'total_hours':1}}])

elapsed_recording = pd.DataFrame(list(elapse))
elapsed_recording.sort(columns='_id')

,_id,number_files,total_hours
3,P100,3,5.353889
13,P101,8,37.093333
0,P102,2,0.517500
6,P103,5,24.591944
10,P105,6,30.273889
14,P106,11,40.318056
2,P107,1,3.299167
1,P108,1,2.430833
7,P109,11,26.370833
18,P110,25,124.304167


In [14]:
elapse = log_data.aggregate([{'$match': {'file_type':'editedRT'}},
                    {'$project':{'patientID':1, 'end_time':1, 'start_time':1, 'elapse_time':{'$subtract':['$end_time', '$start_time']}}},                  
                    {'$group': {'_id': '$patientID', 'total_time':{'$sum':'$elapse_time'}, 
                                'number_files':{'$sum':1}}},
                    {'$project':{'_id':1, 'number_files':1, 'total_hours':{'$divide':['$total_time', 1000*60*60]}}},
                    {'$sort':{'total_hours':1}}])

elapsed_recording = pd.DataFrame(list(elapse))
elapsed_recording.sort(columns='_id')

,_id,number_files,total_hours
5,P100,1,108.150000
17,P101,1,495.700000
7,P102,1,223.416667
10,P103,1,253.716667
14,P104,1,383.450000
20,P105,1,717.700000
2,P106,1,84.433333
1,P107,1,63.083333
3,P108,1,92.883333
12,P109,1,288.500000


In [15]:
elapse = log_data.aggregate([{'$match': {'file_type':'editedRN'}},
                    {'$project':{'patientID':1, 'end_time':1, 'start_time':1, 'elapse_time':{'$subtract':['$end_time', '$start_time']}}},                  
                    {'$group': {'_id': '$patientID', 'total_time':{'$sum':'$elapse_time'}, 
                                'number_files':{'$sum':1}}},
                    {'$project':{'_id':1, 'number_files':1, 'total_hours':{'$divide':['$total_time', 1000*60*60]}}},
                    {'$sort':{'total_hours':1}}])

elapsed_recording = pd.DataFrame(list(elapse))
elapsed_recording.sort(columns='_id')

,_id,number_files,total_hours
5,P100,1,108.150000
17,P101,1,495.700000
7,P102,1,223.416667
10,P103,1,253.716667
15,P104,1,383.450000
20,P105,1,717.700000
2,P106,1,84.433333
1,P107,1,63.083333
3,P108,1,92.883333
12,P109,1,289.250000


In [16]:
elapse = log_data.aggregate([{'$match': {'file_type':'editLab'}},
                    {'$project':{'patientID':1, 'end_time':1, 'start_time':1, 'elapse_time':{'$subtract':['$end_time', '$start_time']}}},                  
                    {'$group': {'_id': '$patientID', 'total_time':{'$sum':'$elapse_time'}, 
                                'number_files':{'$sum':1}}},
                    {'$project':{'_id':1, 'number_files':1, 'total_hours':{'$divide':['$total_time', 1000*60*60]}}},
                    {'$sort':{'total_hours':1}}])

elapsed_recording = pd.DataFrame(list(elapse))
elapsed_recording.sort(columns='_id')

,_id,number_files,total_hours
4,P100,1,97.483333
15,P101,1,364.550000
6,P102,1,221.116667
18,P103,1,594.250000
16,P104,1,375.000000
20,P105,1,710.916667
2,P106,1,80.583333
0,P107,1,27.516667
3,P108,1,89.750000
11,P109,1,279.250000
